In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch

In [ ]:
import numpy as np
import cv2
import mat73
import numpy as np
import cv2
import os
import scipy.io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from matplotlib.widgets import RectangleSelector
import math

In [ ]:
np.random.seed(42)

folder_dir = 'data_np/'

mat_files = sorted([file for file in os.listdir(folder_dir) if file.endswith('.npy')])
idx = 2

print(mat_files[idx])

In [ ]:
image_sequence = np.load(folder_dir + mat_files[idx])
image_sequence = image_sequence[135:, :, :]
roi_coords = []

In [ ]:
# y1, y2, x1, x2 = 68, 320, 160, 423
# image_sequence = image_sequence[y1:y2, x1:x2, :]

image_sequence = image_sequence[:, 105:500, :] 

h, w, t = image_sequence.shape
middle_frame = image_sequence[:, :, t // 2]
plt.figure(figsize=(6, 6))
plt.imshow(middle_frame, cmap='jet')
plt.title('Middle Frame of ROI')
plt.axis('off')
plt.colorbar()
plt.show()

In [ ]:
h, w, sequence_size = image_sequence.shape
plt.figure(figsize=(6, 3))
plt.plot(image_sequence.reshape(h * w, sequence_size)[5000,173:300])
plt.grid(True)
plt.tight_layout()
plt.show()

diff = np.diff(image_sequence.reshape(h * w, sequence_size)[5000,:1000])
t_pulse = np.argmax(diff)
print(t_pulse)
t_pulse = 87

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt

h, w, sequence_size = image_sequence.shape
imageMatrix = image_sequence.reshape(h * w, sequence_size)
imageMatrix = imageMatrix[:, t_pulse:]

imageMatrix = (imageMatrix - np.mean(imageMatrix, axis=1, keepdims=True))

n_components = 10
latent_dim = n_components

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt

h, w, sequence_size = image_sequence.shape
imageMatrix = image_sequence.reshape(h * w, sequence_size)
imageMatrix = imageMatrix[:, t_pulse:]

imageMatrix = (imageMatrix - np.mean(imageMatrix, axis=1, keepdims=True))

n_components = 10
latent_dim = n_components

In [ ]:
imageMatrix = imageMatrix - np.mean(imageMatrix, axis=1, keepdims=True)

U, S, Vh = np.linalg.svd(imageMatrix, full_matrices=False)

Vh = Vh[:n_components, :]

principal_comp = np.matmul(imageMatrix, Vh.T)

pca_array = principal_comp.reshape(h, w, n_components)
pca_array = pca_array.transpose(2, 0, 1)

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    ax.imshow(pca_array[i], cmap='gray')
    ax.set_title(f'PC {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    ax.imshow(pca_array[i], cmap='jet')
    ax.set_title(f'PC {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.decomposition import PCA

C, H, W = pca_array.shape

flat = pca_array.reshape(C, -1).T
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(flat)

explained_var_ratio = pca.explained_variance_ratio_

pca_images = pca_result.T.reshape(n_components, H, W)

eps = 1e-8

combined = np.average(pca_images, axis=0, weights=explained_var_ratio)

combined -= combined.min()
combined /= (combined.max() + eps)
combined = (combined * 255).astype(np.uint8)

import cv2
img_rgb = cv2.cvtColor(combined, cv2.COLOR_GRAY2RGB)

import matplotlib.pyplot as plt
plt.figure(figsize=(3, 3))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch, re
from PIL import Image

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", trust_remote_code=True)
model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto")
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
model.eval()

In [ ]:
from PIL import Image, ImageDraw

image = Image.fromarray(img_rgb).convert("RGB")
image = image.resize((1000, 1000))
W, H = image.size

save_path = "temp_image.jpg"
image.save(save_path)

In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are an expert NDT thermography inspection assistant.\n"
            "Your task is to detect subsurface defects in thermal images and return their exact bbox.\n"
            "Strict output rules:\n"
            "1) Output a JSON list of objects in this format: [{\"label\": \"defect\", \"box\": [x1, y1, x2, y2]}]."
            "2) Coordinates must be integers. (x1, y1) is top-left, (x2, y2) is bottom-right, and x2 > x1, y2 > y1.\n"
            "3) The input image has been resized to 1000×1000 pixels. All bounding boxes must be based on this resolution.\n"
            "4) Make sure the boxes are tight and correct.\n"
            "5) If multiple defects are present, return each on a new line in the same JSON format.\n"
            "6) If no defect is found, return exactly: [{\"label\": \"none\", \"box\": [0, 0, 0, 0]}]\n"
            "7) Do not include any explanation or extra text. Return only valid JSON.\n"
        )
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{save_path}"},
            {"type": "text", "text": "Inspect the thermal image for defects. Return only the required format."}
        ]
    }
]

In [ ]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [ ]:
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

In [ ]:
import json
import re
from PIL import ImageDraw, ImageFont
import matplotlib.pyplot as plt

raw = output_text[0]
cleaned = re.sub(r"```json|```", "", raw).strip()
boxes = json.loads(cleaned)

w, h = image.size
scale_x = w / 1000.0
scale_y = h / 1000.0

draw = ImageDraw.Draw(image)

for item in boxes:
    label = item["label"]
    x1, y1, x2, y2 = item["box"]
    x1 = int(x1 * scale_x)
    x2 = int(x2 * scale_x)
    y1 = int(y1 * scale_y)
    y2 = int(y2 * scale_y)

    draw.rectangle([(x1, y1), (x2, y2)], outline="lime", width=4)
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()
    draw.text((x1 + 5, max(0, y1 - 20)), label, fill="lime", font=font)

plt.figure(figsize=(3, 3))
plt.imshow(image)
plt.axis("off")
plt.show()